In [2]:
# Import modules
from __future__ import print_function

import os
import sys
import gzip
import copy
import shutil
import struct
import itertools
import numpy as np
from random import shuffle
from random import randint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

# Config matplotlib for inline plotting
%matplotlib inline

In [3]:
## Dataloader Function Definitions
def loadData(url, nimage):
    print ('Downloading ' + url)
    gzfile_name, h = urlretrieve(url, './delete.me')
    print ('Done.')
    try:
        with gzip.open(gzfile_name) as gz:
            n = struct.unpack('I', gz.read(4))
            n = struct.unpack('>I', gz.read(4))[0]
            nrow = struct.unpack('>I', gz.read(4))[0]
            ncol = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(nimage * nrow * ncol), dtype = np.uint8)
    finally:
        os.remove(gzfile_name)
    return res.reshape((nimage, nrow * ncol))

def loadLabels(url, nimage):
    print ('Downloading ' + url)
    gzfile_name, h = urlretrieve(url, './delete.me')
    print ('Done.')
    try:
        with gzip.open(gzfile_name) as gz:
            n = struct.unpack('I', gz.read(4))
            n = struct.unpack('>I', gz.read(4))
            res = np.frombuffer(gz.read(nimage), dtype = np.uint8)
    finally:
        os.remove(gzfile_name)
    return res.reshape((nimage, 1))

def download(dataurl, labelsurl, nimage):
    data = loadData(dataurl, nimage)
    labels = loadLabels(labelsurl, nimage)
    return np.hstack((data, labels))

In [4]:
## Download Data From Remote Source
# URLs for the train image and label data
train_image_url = 'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
train_labels_url = 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
num_train_samples = 60000
# Download train data
print("Downloading train data")
train = download(train_image_url, train_labels_url, num_train_samples)
# URLs for the test image and label data
test_image_url = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
test_labels_url = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
num_test_samples = 10000
# Download test data
print("Downloading test data")
test = download(test_image_url, test_labels_url, num_test_samples)

## Prepare Train and Test data
# Separate X and Y from data
Xtrain = train[:,0:-1]
Xtest = test[:,0:-1]
Ytrain = train[:,-1]
Ytest = test[:,-1]
Xtrain = Xtrain.reshape(60000,28,28)
Xtest = Xtest.reshape(10000,28,28)

Done.
Done.
Done.
Done.


In [5]:
## Hyperparameters for the layers and filters
num_digits = 10 
l_filter = 5
l_filter = 5
num_filters = 5
l_output_map = 28-l_filter+1 

## Initialize the parameters for the Convolutional Neural Network
parameters = {
    'K': np.random.randn(l_filter, l_filter, num_filters)/np.sqrt(l_filter*l_filter),
    'W': np.random.randn(num_digits, l_output_map, l_output_map, num_filters)/np.sqrt(l_output_map*l_output_map),
    'b': np.random.randn(num_digits) 
}
parameters_grads = copy.deepcopy(parameters)

## Define the iterable tables for use in the conv function
i1= range(l_output_map)
i2 = range(l_output_map)
i3= range(l_filter)
i4 = range(l_filter)
i5 = range(num_filters)
iteratable_forward = list(itertools.product(i1, i2, i5))
iteratable_backward= list(itertools.product(i3, i4, i5))

In [14]:
## Define the softmax function
def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))

## Define the convolution function
def conv(X, K, iteratable, l_f):
    conv_Z = np.array([np.tensordot(K[:,:,ijk[2]],X[ijk[0]:ijk[0]+l_f,ijk[1]:ijk[1]+l_f], axes=((0,1), (0,1))) for ijk in iteratable])
    return conv_Z

## Define the activation function tanh
def tanh(M):
    return np.tanh(M)

## Define the derivative of the activation function
def tanh_p(M):
    return 1.0-np.tanh(M)**2

## Define the feed forward function for computing intermediate layers
def feedforward(x, y): 
    Z = conv(x, parameters['K'], iteratable_forward, l_filter).reshape(l_output_map, l_output_map, num_filters) 
    H = tanh(Z)
    U = np.tensordot(parameters['W'], H, axes = ((1,2,3),(0,1,2))) + parameters['b']
    f = softmax(U)
    return Z, H, f

## Define the backpropogation function for updating parameters
def backpropagate(x, y, Z, H, f):
    drdU = -1.0*f
    drdU[y] = (drdU[y] + 1.0) 
    drdU = -drdU
    drdb = drdU
    drdW = np.tensordot(drdU.T, H, axes = 0)[0]
    delta = np.tensordot(drdU.T, parameters['W'], axes = 1)[0] 
    drdK =  conv(x, np.multiply(delta, tanh_p(Z)), iteratable_backward, 28-l_filter+1).reshape(l_filter, l_filter, num_filters)
    return drdb,drdW,drdK
        
## Predict function used to compute accuracy on a dataset
def predict(Xdata, Ydata):
    total_correct = 0
    for index in range(len(Xdata)):
        y = Ydata[index]
        x = Xdata[index][:]
        _, _, p = feedforward(x, y)
        prediction = np.argmax(p)
        if (prediction == y): 
            total_correct += 1
    accuracy = total_correct/np.float(len(Xdata))
    return accuracy

In [12]:
## Define parameters for training
lr = 1e-2
num_epoch = 20

## Implement Training with SGD
for epoch in range(num_epoch):
    if (epoch > 5):
        lr = 1e-3
    if (epoch > 10):
        lr = 1e-4
    if (epoch > 15):
        lr = 1e-5
        
    # Do Feedforward and Backpropagate, and update parameters according to SGD 
    num_correct_predictions = 0
    for n in range(len(Xtrain)):
        if (n%100 == 0):
            print("Iteration #", n, "completed out of ", len(Xtrain),"iterations")
        n_random = randint(0,len(Xtrain)-1)
        y = Ytrain[n_random]
        x = Xtrain[n_random][:]
        Z, H, f = feedforward(x, y)
        prediction = np.argmax(f)
        if (prediction == y):
            num_correct_predictions += 1
        drdb,drdW,drdK = backpropagate(x, y, Z, H, f)
        parameters['W'] -= lr*drdW
        parameters['b'] -= lr*drdb
        parameters['K'] -= lr*drdK
    print('For epoch ', epoch,'/num_epoch', ', accuracy in training set = ', num_correct_predictions/len(Xtrain))

## Compute the test accuracy 
test_accuracy = predict(Xtest, Ytest)
print('Accuracy in testing set =', test_accuracy)

Iteration # 0 completed out of  60000 iterations
Iteration # 100 completed out of  60000 iterations
Iteration # 200 completed out of  60000 iterations
Iteration # 300 completed out of  60000 iterations
Iteration # 400 completed out of  60000 iterations
Iteration # 500 completed out of  60000 iterations
Iteration # 600 completed out of  60000 iterations
Iteration # 700 completed out of  60000 iterations
Iteration # 800 completed out of  60000 iterations
Iteration # 900 completed out of  60000 iterations


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration # 1000 completed out of  60000 iterations
Iteration # 1100 completed out of  60000 iterations
Iteration # 1200 completed out of  60000 iterations
Iteration # 1300 completed out of  60000 iterations
Iteration # 1400 completed out of  60000 iterations
Iteration # 1500 completed out of  60000 iterations
Iteration # 1600 completed out of  60000 iterations
Iteration # 1700 completed out of  60000 iterations
Iteration # 1800 completed out of  60000 iterations
Iteration # 1900 completed out of  60000 iterations
Iteration # 2000 completed out of  60000 iterations
Iteration # 2100 completed out of  60000 iterations
Iteration # 2200 completed out of  60000 iterations
Iteration # 2300 completed out of  60000 iterations
Iteration # 2400 completed out of  60000 iterations
Iteration # 2500 completed out of  60000 iterations
Iteration # 2600 completed out of  60000 iterations
Iteration # 2700 completed out of  60000 iterations
Iteration # 2800 completed out of  60000 iterations
Iteration # 

Iteration # 16700 completed out of  60000 iterations
Iteration # 16800 completed out of  60000 iterations
Iteration # 16900 completed out of  60000 iterations
Iteration # 17000 completed out of  60000 iterations
Iteration # 17100 completed out of  60000 iterations
Iteration # 17200 completed out of  60000 iterations
Iteration # 17300 completed out of  60000 iterations
Iteration # 17400 completed out of  60000 iterations
Iteration # 17500 completed out of  60000 iterations
Iteration # 17600 completed out of  60000 iterations
Iteration # 17700 completed out of  60000 iterations
Iteration # 17800 completed out of  60000 iterations
Iteration # 17900 completed out of  60000 iterations
Iteration # 18000 completed out of  60000 iterations
Iteration # 18100 completed out of  60000 iterations
Iteration # 18200 completed out of  60000 iterations
Iteration # 18300 completed out of  60000 iterations
Iteration # 18400 completed out of  60000 iterations
Iteration # 18500 completed out of  60000 iter

Iteration # 32200 completed out of  60000 iterations
Iteration # 32300 completed out of  60000 iterations
Iteration # 32400 completed out of  60000 iterations
Iteration # 32500 completed out of  60000 iterations
Iteration # 32600 completed out of  60000 iterations
Iteration # 32700 completed out of  60000 iterations
Iteration # 32800 completed out of  60000 iterations
Iteration # 32900 completed out of  60000 iterations
Iteration # 33000 completed out of  60000 iterations
Iteration # 33100 completed out of  60000 iterations
Iteration # 33200 completed out of  60000 iterations
Iteration # 33300 completed out of  60000 iterations
Iteration # 33400 completed out of  60000 iterations
Iteration # 33500 completed out of  60000 iterations
Iteration # 33600 completed out of  60000 iterations
Iteration # 33700 completed out of  60000 iterations
Iteration # 33800 completed out of  60000 iterations
Iteration # 33900 completed out of  60000 iterations
Iteration # 34000 completed out of  60000 iter

Iteration # 47700 completed out of  60000 iterations
Iteration # 47800 completed out of  60000 iterations
Iteration # 47900 completed out of  60000 iterations
Iteration # 48000 completed out of  60000 iterations
Iteration # 48100 completed out of  60000 iterations
Iteration # 48200 completed out of  60000 iterations
Iteration # 48300 completed out of  60000 iterations
Iteration # 48400 completed out of  60000 iterations
Iteration # 48500 completed out of  60000 iterations
Iteration # 48600 completed out of  60000 iterations
Iteration # 48700 completed out of  60000 iterations
Iteration # 48800 completed out of  60000 iterations
Iteration # 48900 completed out of  60000 iterations
Iteration # 49000 completed out of  60000 iterations
Iteration # 49100 completed out of  60000 iterations
Iteration # 49200 completed out of  60000 iterations
Iteration # 49300 completed out of  60000 iterations
Iteration # 49400 completed out of  60000 iterations
Iteration # 49500 completed out of  60000 iter

KeyboardInterrupt: 

In [17]:
## Compute the test accuracy 
test_accuracy = predict(Xtest, Ytest)
print('Accuracy in testing set =', test_accuracy)

KeyboardInterrupt: 

In [16]:
print(num_correct_predictions)

192
